In [ ]:
# Tests for base Horizon functionality
import os
import sys
from datetime import date
import warnings
import numpy as np

warnings.filterwarnings('ignore')

sys.path.append('../../../seismiqb')

from seismiqb import Field, Horizon

In [ ]:
# Workspace constants
TESTS_ROOT_DIR = './'

# Visualization parameters
SCALE = 1
SHOW_FIGURES = True

In [ ]:
# Tests parameters
OUTPUT_DIR = os.path.join(TESTS_ROOT_DIR, "horizon_test_files")

CUBE_PATH = os.path.join(OUTPUT_DIR, f"test_cube.sgy")
HORIZON_PATH = os.path.join(OUTPUT_DIR, f"test_horizon")

In [ ]:
def test_horizons_equality(horizon_1, horizon_2, test_name):
    " Check two horizons equality."
    assert np.array_equal(horizon_1.full_matrix, horizon_2.full_matrix), f"{test_name} test failed"
    assert np.array_equal(horizon_1.points, horizon_2.points), f"{test_name} test failed"

# Initialization

In [ ]:
%%time
field = Field(CUBE_PATH)

# init from_file
horizon = Horizon(HORIZON_PATH, field=field)

horizon.show(show=SHOW_FIGURES, scale=SCALE)

In [ ]:
%%time
# from_points init
new_horizon = Horizon(storage=horizon.points, field=horizon.field, name="tester")

test_horizons_equality(horizon, new_horizon, test_name="from_points init")

In [ ]:
%%time
# from_matrix init
new_horizon = Horizon(storage=horizon.matrix, i_min=horizon.i_min, x_min=horizon.x_min,
                      field=horizon.field, name="tester")

test_horizons_equality(horizon, new_horizon, test_name="from_matrix init")

In [ ]:
%%time
# from_full_matrix init
new_horizon = Horizon(storage=horizon.full_matrix,
                      field=horizon.field, name="tester")

test_horizons_equality(horizon, new_horizon, test_name="from_full_matrix init")

In [ ]:
%%time
# from_dict
horizon_dict = {}
for k, v in zip(horizon.points[:, :2], horizon.points[:, 2]):
    horizon_dict[tuple(k)] = v

new_horizon = Horizon(storage=horizon_dict, transform=False, field=horizon.field, name="tester")

test_horizons_equality(horizon, new_horizon, test_name="from_dict init")

# Dump

In [ ]:
tmp_horizon_path = os.path.join(OUTPUT_DIR, "tmp_horizon")

In [ ]:
%%time
# dump
horizon.dump(path=tmp_horizon_path)

dumped_horizon = Horizon(storage=tmp_horizon_path, field=field)

test_horizons_equality(horizon, dumped_horizon, test_name="dump")

os.remove(tmp_horizon_path)

In [ ]:
%%time
# dump_float
depths_diff_threshold = 2
horizon.dump_float(path=tmp_horizon_path, depths_diff_threshold=depths_diff_threshold)

dumped_horizon = Horizon(storage=tmp_horizon_path, field=field)

assert np.all(
    np.isclose(horizon.points, dumped_horizon.points, atol=depths_diff_threshold)
), "dump_float test failed"

os.remove(tmp_horizon_path)

# Base methods

In [ ]:
%%time
horizon_points_from_matrix = Horizon.matrix_to_points(matrix=horizon.matrix)

# Shift (it exists in methods that call `matrix_to_points`)
horizon_points_from_matrix[:, 0] += horizon.i_min
horizon_points_from_matrix[:, 1] += horizon.x_min

assert np.array_equal(horizon.points, horizon_points_from_matrix), "matrix_to_points test failed"

In [ ]:
%%time
horizon_matrix_from_points = Horizon.points_to_matrix(
    points=horizon.points, i_min=horizon.i_min, x_min=horizon.x_min,
    i_length=horizon.i_length, x_length=horizon.x_length
)

assert np.array_equal(horizon.matrix, horizon_matrix_from_points), "points_to_matrix test failed"

# Evaluate

In [ ]:
%%time
_ = horizon.evaluate(scale=SCALE, bad_color='black')

# Get values

In [ ]:
%%time
axis = 0
loc = horizon.shape[axis] // 2
_ = horizon.load_slide(loc=loc, axis=axis)

horizon.show_slide(show=SHOW_FIGURES, loc=loc, axis=axis, scale=SCALE)

# Visualizations

In [ ]:
%%time
# Attributes visualizations are in another notebook
horizon.show(scale=SCALE)

# Show slide
axis = 0
loc = horizon.shape[axis] // 2

horizon.show_slide(loc=loc, axis=axis, width=5, separate=True,
                   cmap=['gray', 'viridis'], grid=False, scale=SCALE)